In [1]:
import numpy as np

In [2]:
!ls

MNIST-2-layer.-reference.ipynb mnist_train_labels.npy
mnist_test_images.npy          mnist_validation_images.npy
mnist_test_labels.npy          mnist_validation_labels.npy
mnist_train_images.npy


In [3]:
X_train = np.load("mnist_train_images.npy")
y_train = np.load("mnist_train_labels.npy")
X_test = np.load("mnist_test_images.npy")
y_test = np.load("mnist_test_labels.npy")
X_validation = np.load("mnist_validation_images.npy")
y_validation = np.load("mnist_validation_labels.npy")

In [4]:
w = np.random.random([X_train.shape[1], y_train.shape[1]])
b = np.random.random([1,y_train.shape[1]])



model = {
    "w": w,
    "b": b
}    

In [5]:
def softmax(z):
#     z -= np.max(z)
    return (np.exp(z).T / np.sum(np.exp(z), axis=1))

def forward_pass(x, model):
    w= model["w"]
    b= model["b"]
    z = (x@w+b)
    a = softmax(z).T
#     y = np.argmax(a, axis=1)
    return a

def calculate_CE_loss(a, one_hot_y):
    cost = (-1 / a.shape[0]) * np.sum(one_hot_y * np.log(a))
    return cost

def calculate_regularization(w,alpha):
    REGULARIZATION = 0.5 * alpha * (w.T@w) 
    return REGULARIZATION
def backprop(prediction, x, y, model, alpha=0.01, lr=0.00001):
    w= model["w"]
    b= model["b"]
    grad_w = (-1/(x.shape[0]))*(x.T@(y-prediction)) + alpha * w
    w = w - lr*grad_w
    grad_b = -1*np.mean(y-prediction, axis = 0)
    b = b - lr*grad_b
    model["w"] = w
    model["b"] = b
    return model
def evaluate(x = X_validation, y = y_validation, model=model, alpha=0.1):
    prediction = forward_pass(x, model)
    CE_loss = calculate_CE_loss(prediction, y)
    w= model["w"]
    reg_loss = calculate_regularization(w,alpha)
    return CE_loss+reg_loss
def accuracy(y, y_true):
    return (sum(y == y_true)/len(y_true)) * 100

In [6]:
# Hyperparameters - GRID SEARCH
lrs = [3.3e-2]
alphas = [0.001]
batch_sizes = [10]
epochs = [100]

In [7]:
def relu(x):
    activated = np.maximum(0, x) #just for clarification
    return activated
def reluDerivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [149]:
i = 0
batch_size = 1000
data_x, data_y = X_train[(i)*batch_size:(i)*batch_size+batch_size,], y_train[(i)*batch_size:(i)*batch_size+batch_size,]



In [ ]:
def softmax(z):
    z -= np.max(z)
    return (np.exp(z).T / np.sum(np.exp(z), axis=1))


many_neurons_1 = 75
w1 = np.random.random([data_x.shape[1], many_neurons_1])/10
b1 = np.random.random([1,many_neurons_1])/10

many_neurons_2 = 50
w2 = np.random.random([many_neurons_1, many_neurons_2])/10
b2 = np.random.random([1,many_neurons_2])/10

# many_neurons_2 = 50
w3 = np.random.random([many_neurons_2, data_y.shape[1]])/10
b3 = np.random.random([1,data_y.shape[1]])/10

## 2 hidden layer model

In [ ]:
many_neurons_1 = 75
w1 = np.random.random([data_x.shape[1], many_neurons_1])/10
b1 = np.random.random([1,many_neurons_1])/10

many_neurons_2 = 50
w2 = np.random.random([many_neurons_1, many_neurons_2])/10
b2 = np.random.random([1,many_neurons_2])/10

# many_neurons_2 = 50
w3 = np.random.random([many_neurons_2, data_y.shape[1]])/10
b3 = np.random.random([1,data_y.shape[1]])/10

In [ ]:
lrs = [3.3e-2]
alphas = [0.001]
batch_sizes = [100]
epochs = [200]

for lr in lrs:
    for alpha in alphas:
        for batch_size in batch_sizes:
            for epoch in epochs:
                batches = (int)(len(X_train)/batch_size)
                for _ in range(epoch):
                    for i in range(batches):
                        data_x, data_y = X_train[(i)*batch_size:(i)*batch_size+batch_size,], y_train[(i)*batch_size:(i)*batch_size+batch_size,]
                        z1 = np.dot(data_x,w1)+b1
                        a1 = relu(z1)
                        z2 = np.dot(a1,w2)+b2
                        a2 = relu(z2)
                        z3 = np.dot(a2,w3)+b3
                        a3 = softmax(z3).T

                        batch = len(data_x)
                        g = (np.subtract(a3,data_y))
                        db3 = np.mean(g, axis = 0) 
                        dw3 = (1/batch)* np.dot(a2.T,g)
                        dw3 += alpha * w3

                        g = np.multiply(np.dot(g,w3.T),reluDerivative(z2))
                        db2 = np.mean(g, axis = 0)
                        dw2 = (1/batch)* np.dot(a1.T,g)
                        dw2 += alpha * w2
                        
                        g = np.multiply(np.dot(g,w2.T),reluDerivative(z1))
                        db1 = np.mean(g, axis = 0)
                        dw1 = (1/batch)* np.dot(data_x.T,g)
                        dw1 +=  alpha * w1
                        
                        lr = 1e-2
                        w3 -= lr*dw3
                        w2 -= lr*dw2
                        w1 -= lr*dw1

                        b3 -= lr*db3
                        b2 -= lr*db2
                        b1 -= lr*db1

## Validation data - check accuracy

In [ ]:
z1 = np.dot(X_validation,w1)+b1
a1 = relu(z1)
z2 = np.dot(a1,w2)+b2
a2 = relu(z2)
z3 = np.dot(a2,w3)+b3
a3 = softmax(z3).T
output = np.argmax(a3, axis=1)
actual_y = np.argmax(y_validation, axis=1)
validation_accuracy = accuracy(output,actual_y)
print(validation_accuracy) 

## Test data - check accuracy

In [ ]:
z1 = np.dot(X_test,w1)+b1
a1 = relu(z1)
z2 = np.dot(a1,w2)+b2
a2 = relu(z2)
z3 = np.dot(a2,w3)+b3
a3 = softmax(z3).T
output = np.argmax(a3, axis=1)
actual_y = np.argmax(y_test, axis=1)
validation_accuracy = accuracy(output,actual_y)
print(validation_accuracy) 

## Multilayer - How to do it??

In [126]:
def init_params(input_size = 784, output_size=10, many_hidden_layers=4):
    params = {}
    nodes_in_layers = [60, 50, 55, 45, 30, 25, 20]
    for layer in range(many_hidden_layers+1):
        if(layer==0):
            params["w" + str(layer)]=np.random.random([input_size, nodes_in_layers[layer]])/20        
        elif(layer==(many_hidden_layers)):
            params["w" + str(layer)]=np.random.random([nodes_in_layers[layer-1], output_size])/20
        else:
            params["w" + str(layer)] = np.random.random([nodes_in_layers[layer-1], nodes_in_layers[layer]])/20

        if(layer==0):
            params["b" + str(layer)]=np.random.random([1, nodes_in_layers[layer]])/20
        elif(layer==(many_hidden_layers)):
            params["b" + str(layer)]=np.random.random([1, output_size])/20
        else:
            params["b" + str(layer)] = np.random.random([1, nodes_in_layers[layer]])/20
    return params

In [187]:
# forward pass with unknown depth
# hey bhagwan!!
def forward_pass(data_x, params, many_hidden_layers):
    layer_cache = {}
    z_current = data_x
    for i in range(many_hidden_layers+1):
    #     affine transformation
        z_current = np.dot(z_current,params["w" + str(i)]) + params["b" + str(i)]
    #     activation 
        if(i == many_hidden_layers):
            a_current = softmax(z_current).T
        else:
            a_current = relu(z_current)
#         print("z mean = ", z_current.mean())
#         print("a mean = ",a_current.mean())
        layer_cache["z" + str(i)] = z_current;
        layer_cache["a" + str(i)] = a_current;
    return layer_cache




In [128]:
def calculate_derivatives(data_x, data_y,params, layer_cache, many_hidden_layers, alpha = 0.001):
    derivatives = {}
    batch = len(data_x)
    for i in reversed(range(many_hidden_layers+1)):
        if(i==many_hidden_layers):
            g = (np.subtract(layer_cache["a" + str(i)],data_y))
        else:
            g = np.multiply(np.dot(g,params["w" + str(i+1)].T),reluDerivative(layer_cache["z" + str(i)]))
        derivatives["b" + str(i)] = np.mean(g, axis = 0) 
        if(i==0):
            derivatives["w" + str(i)] = (1/batch)* np.dot(data_x.T,g)
#             derivatives["w" + str(i)] += alpha * params["w" + str(i)]
        else:
            derivatives["w" + str(i)] = (1/batch)* np.dot(layer_cache["a" + str(i-1)].T,g)
#             derivatives["w" + str(i)] += alpha * params["w" + str(i)]
    return derivatives

In [129]:
# update weights

def update_weights(params, derivatives, many_hidden_layers,lr = 1e-3):
    for i in range(many_hidden_layers+1):
        params["w" + str(i)] -= lr*derivatives["w" + str(i)]
        params["b" + str(i)] -= lr*derivatives["b" + str(i)]
    return params

In [164]:
params = init_params(input_size = 784, output_size=10, many_hidden_layers=4)

In [159]:
many_hidden_layers = 4
for _ in range(1000):
    layer_cache = forward_pass(data_x, params, many_hidden_layers)
    derivatives = calculate_derivatives(data_x, data_y,params, layer_cache, many_hidden_layers, alpha = 0.001)
    params = update_weights(params, derivatives, many_hidden_layers,lr = 1e-2)

y_hat = layer_cache["a"+str(many_hidden_layers)]
# x_current = softmax(y_hat).T
output = np.argmax(y_hat, axis=1)
actual_y = np.argmax(data_y, axis=1)
validation_accuracy = accuracy(output,actual_y)
print(validation_accuracy) 

45.9


In [160]:
layer_cache = forward_pass(X_validation, params, many_hidden_layers)
y_hat = layer_cache["a"+str(many_hidden_layers)]
# x_current = softmax(y_hat).T
output = np.argmax(y_hat, axis=1)
actual_y = np.argmax(y_validation, axis=1)
validation_accuracy = accuracy(output,actual_y)
print(validation_accuracy) 

40.82


In [182]:
params = init_params(input_size = 784, output_size=10, many_hidden_layers=4)

In [184]:
lrs = [3.3e-3]
alphas = [0.001]
batch_sizes = [100]
epochs = [26]

for lr in lrs:
    for alpha in alphas:
        for batch_size in batch_sizes:
            for epoch in epochs:
                batches = (int)(len(X_train)/batch_size)
                for _ in range(epoch):
                    for i in range(batches):
                        data_x, data_y = X_train[(i)*batch_size:(i)*batch_size+batch_size,], y_train[(i)*batch_size:(i)*batch_size+batch_size,]
                        layer_cache = forward_pass(data_x, params, many_hidden_layers)
                        derivatives = calculate_derivatives(data_x, data_y,params, layer_cache, many_hidden_layers, alpha = 0.0)
                        params = update_weights(params, derivatives, many_hidden_layers,lr = 3.3e-3)

                layer_cache = forward_pass(X_validation, params, many_hidden_layers)
                y_hat = layer_cache["a"+str(many_hidden_layers)]
                # x_current = softmax(y_hat).T
                output = np.argmax(y_hat, axis=1)
                actual_y = np.argmax(y_validation, axis=1)
                validation_accuracy = accuracy(output,actual_y)
                print(validation_accuracy)    

42.08


In [172]:
layer_cache = forward_pass(X_validation, params, many_hidden_layers)
y_hat = layer_cache["a"+str(many_hidden_layers)]
# x_current = softmax(y_hat).T
output = np.argmax(y_hat, axis=1)
actual_y = np.argmax(y_validation, axis=1)
validation_accuracy = accuracy(output,actual_y)
print(validation_accuracy) 

9.58
